# BOP Data Analysis - Silver Layer

Analysis of Balance of Payments (BOP) data using PySpark.

This notebook performs basic exploratory analysis on BOP data.

## 1. Environment Setup

In [13]:
# Configure environment for local execution
import os
import sys

if 'AIRFLOW_HOME' not in os.environ:
    # Running locally - add PySpark from user installation to path
    pyspark_paths = [
        '/home/davi/.local/lib/python3.12/site-packages',  # Python 3.12 user site
        '/home/davi/.local/lib/python3.11/site-packages',  # Python 3.11 user site
        '/usr/local/lib/python3.12/dist-packages',         # System Python 3.12
        '/usr/local/lib/python3.11/dist-packages',         # System Python 3.11
    ]
    
    pyspark_found = False
    for path in pyspark_paths:
        if os.path.exists(os.path.join(path, 'pyspark')):
            if path not in sys.path:
                sys.path.insert(0, path)
            pyspark_found = True
            print(f"✅ PySpark found at: {path}")
            break
    
    if not pyspark_found:
        print("⚠️  PySpark not found. Installing...")
        import subprocess
        # Install for the current Python version
        subprocess.check_call([sys.executable, "-m", "pip", "install", "--user", "pyspark==3.5.1", "pyarrow==15.0.2", "findspark==2.0.1"])
        print("✅ PySpark installed successfully!")
    
    # Verify import
    try:
        import pyspark
        print(f"✅ PySpark {pyspark.__version__} loaded successfully")
    except ImportError as e:
        print(f"❌ Error importing PySpark: {e}")
    
    # Configure Java
    java_home = '/usr/lib/jvm/java-17-openjdk-amd64'
    if os.path.exists(java_home):
        os.environ['JAVA_HOME'] = java_home
        print(f"✅ Java configured: {java_home}")
    else:
        print("⚠️  Warning: Java 17 not found at expected location")
        print("   Install with: sudo apt install openjdk-17-jdk")
else:
    print("🐳 Running in Airflow - environment already configured")

✅ PySpark found at: /home/davi/.local/lib/python3.12/site-packages
✅ PySpark 3.5.1 loaded successfully
✅ Java configured: /usr/lib/jvm/java-17-openjdk-amd64


In [1]:
import sys
import os
from pathlib import Path

# Define paths
if 'AIRFLOW_HOME' in os.environ:
    # Running in Airflow
    BASE_PATH = Path('/opt/airflow')
    DATA_PATH = BASE_PATH / 'base_dados' / 'Resultados'
    SPARK_CONFIG_PATH = BASE_PATH / 'spark_config'
else:
    # Running manually
    BASE_PATH = Path.cwd().parent
    DATA_PATH = BASE_PATH / 'base_dados' / 'Resultados'
    SPARK_CONFIG_PATH = BASE_PATH / 'spark_config'

# Add spark_config to path
sys.path.insert(0, str(SPARK_CONFIG_PATH))

print(f"Base Path: {BASE_PATH}")
print(f"Data Path: {DATA_PATH}")
print(f"Spark Config Path: {SPARK_CONFIG_PATH}")

Base Path: /home/mateus/Documentos/Faculdade/bancos2/TrabalhoSBD2
Data Path: /home/mateus/Documentos/Faculdade/bancos2/TrabalhoSBD2/base_dados/Resultados
Spark Config Path: /home/mateus/Documentos/Faculdade/bancos2/TrabalhoSBD2/spark_config


## 2. Initialize Spark Session

In [2]:
from config import SparkConfig, DataSchemas
from pyspark.sql import functions as F

# Create Spark session
spark_config = SparkConfig(app_name="BOP_Silver_Analysis")
spark = spark_config.create_spark_session()
spark_config.configure_for_banking_data()

print(f"Spark Version: {spark.version}")
print(f"Spark UI: {spark.sparkContext.uiWebUrl}")

25/10/17 12:04:39 WARN Utils: Your hostname, mateus-730QED resolves to a loopback address: 127.0.1.1; using 172.29.99.42 instead (on interface wlo1)
25/10/17 12:04:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/17 12:04:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
INFO:config:Spark Session criada: BOP_Silver_Analysis
INFO:config:Spark UI disponível em: http://172.29.99.42:4040
25/10/17 12:04:44 WARN SQLConf: The SQL config 'spark.sql.adaptive.coalescePartitions.minPartitionNum' has been deprecated in Spark v3.2 and may be removed in the future. Use 'spark.sql.adaptive.coalescePartitions.minPartitionSize' instead.
25/10/17 12:04:44 WARN SQLConf: The SQL config 'spark.sql.adaptive.coalescePartitions.minPartitionNum' has been deprecated in Spark v3

Spark Version: 3.5.1
Spark UI: http://172.29.99.42:4040


## 3. Load BOP Data

In [16]:
# Load BOP data
bop_file = DATA_PATH / 'BOP.csv'
print(f"Loading file: {bop_file}")

df_bop = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(str(bop_file))

print(f"Total records loaded: {df_bop.count():,}")

Loading file: /home/davi/Área de Trabalho/bancos2/TrabalhoSBD2/base_dados/Resultados/BOP.csv


25/10/09 15:16:55 WARN SQLConf: The SQL config 'spark.sql.adaptive.coalescePartitions.minPartitionNum' has been deprecated in Spark v3.2 and may be removed in the future. Use 'spark.sql.adaptive.coalescePartitions.minPartitionSize' instead.
25/10/09 15:16:55 WARN SQLConf: The SQL config 'spark.sql.adaptive.coalescePartitions.minPartitionNum' has been deprecated in Spark v3.2 and may be removed in the future. Use 'spark.sql.adaptive.coalescePartitions.minPartitionSize' instead.
25/10/09 15:16:55 WARN SQLConf: The SQL config 'spark.sql.adaptive.coalescePartitions.minPartitionNum' has been deprecated in Spark v3.2 and may be removed in the future. Use 'spark.sql.adaptive.coalescePartitions.minPartitionSize' instead.


Total records loaded: 180,804


## 4. Data Schema

In [17]:
# Display schema
df_bop.printSchema()

root
 |-- COUNTRY: string (nullable = true)
 |-- BOP_ACCOUNTING_ENTRY: string (nullable = true)
 |-- INDICATOR: string (nullable = true)
 |-- UNIT: string (nullable = true)
 |-- FREQUENCY: string (nullable = true)
 |-- TIME_PERIOD: string (nullable = true)
 |-- value: double (nullable = true)
 |-- index: string (nullable = true)



## 5. Basic Statistics

In [18]:
# Statistical summary
df_bop.describe().show()

25/10/09 15:17:02 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-------+--------------------+---------+------+---------+-----------+--------------------+-----+
|summary|COUNTRY|BOP_ACCOUNTING_ENTRY|INDICATOR|  UNIT|FREQUENCY|TIME_PERIOD|               value|index|
+-------+-------+--------------------+---------+------+---------+-----------+--------------------+-----+
|  count| 180804|              180804|   180804|180804|   180804|     180804|              180804|    0|
|   mean|   NULL|                NULL|     NULL|  NULL|     NULL|       NULL| 8.002790936956046E9| NULL|
| stddev|   NULL|                NULL|     NULL|  NULL|     NULL|       NULL|4.131887512070207E10| NULL|
|    min|    ARG|             A_NFA_T|      CAB|   USD|        Q|    2000-Q1|-6.73055075813792...| NULL|
|    max|    ZAF|          NNAFANIL_T|       SF|   USD|        Q|    2025-Q2|         1.158498E12| NULL|
+-------+-------+--------------------+---------+------+---------+-----------+--------------------+-----+



## 6. Sample Data

In [19]:
# Display sample records
print("Sample records:")
df_bop.show(10, truncate=False)

Sample records:
+-------+--------------------+---------+----+---------+-----------+-----------+-----+
|COUNTRY|BOP_ACCOUNTING_ENTRY|INDICATOR|UNIT|FREQUENCY|TIME_PERIOD|value      |index|
+-------+--------------------+---------+----+---------+-----------+-----------+-----+
|USA    |NETCD_T             |CAB      |USD |Q        |2000-Q1    |-8.4585E10 |NULL |
|USA    |NETCD_T             |CAB      |USD |Q        |2000-Q2    |-9.6104E10 |NULL |
|USA    |NETCD_T             |CAB      |USD |Q        |2000-Q3    |-1.14965E11|NULL |
|USA    |NETCD_T             |CAB      |USD |Q        |2000-Q4    |-1.06274E11|NULL |
|USA    |NETCD_T             |CAB      |USD |Q        |2001-Q1    |-9.6454E10 |NULL |
|USA    |NETCD_T             |CAB      |USD |Q        |2001-Q2    |-9.5128E10 |NULL |
|USA    |NETCD_T             |CAB      |USD |Q        |2001-Q3    |-1.13956E11|NULL |
|USA    |NETCD_T             |CAB      |USD |Q        |2001-Q4    |-8.8541E10 |NULL |
|USA    |NETCD_T             |CAB     

## 7. Data Quality Checks

In [20]:
# Count records by country
print("Records by country:")
df_bop.groupBy("COUNTRY") \
    .count() \
    .orderBy(F.desc("count")) \
    .show(10)

Records by country:
+-------+-----+
|COUNTRY|count|
+-------+-----+
|    KOR| 3978|
|    BRA| 3978|
|    MEX| 3978|
|    ITA| 3939|
|    GBR| 3939|
|    FIN| 3939|
|    NLD| 3939|
|    POL| 3939|
|    CZE| 3939|
|    AUS| 3939|
+-------+-----+
only showing top 10 rows

+-------+-----+
|COUNTRY|count|
+-------+-----+
|    KOR| 3978|
|    BRA| 3978|
|    MEX| 3978|
|    ITA| 3939|
|    GBR| 3939|
|    FIN| 3939|
|    NLD| 3939|
|    POL| 3939|
|    CZE| 3939|
|    AUS| 3939|
+-------+-----+
only showing top 10 rows



In [21]:
# Count records by indicator
print("Records by indicator:")
df_bop.groupBy("INDICATOR") \
    .count() \
    .orderBy(F.desc("count")) \
    .show(10)

Records by indicator:
+---------+-----+
|INDICATOR|count|
+---------+-----+
|      IN1|14430|
|       GS|14430|
|      IN2|14409|
|     O_F2|13852|
|     O_F4|13833|
|       SF|13748|
|    O_F81|12821|
|     P_F5| 9084|
|     P_F3| 8928|
|     D_F5| 8923|
+---------+-----+
only showing top 10 rows

+---------+-----+
|INDICATOR|count|
+---------+-----+
|      IN1|14430|
|       GS|14430|
|      IN2|14409|
|     O_F2|13852|
|     O_F4|13833|
|       SF|13748|
|    O_F81|12821|
|     P_F5| 9084|
|     P_F3| 8928|
|     D_F5| 8923|
+---------+-----+
only showing top 10 rows



In [22]:
# Count records by frequency
print("Records by frequency:")
df_bop.groupBy("FREQUENCY") \
    .count() \
    .orderBy(F.desc("count")) \
    .show()

Records by frequency:
+---------+------+
|FREQUENCY| count|
+---------+------+
|        Q|180804|
+---------+------+

+---------+------+
|FREQUENCY| count|
+---------+------+
|        Q|180804|
+---------+------+



## 8. Value Analysis

In [23]:
# Check for null values
print("Null value counts:")
df_bop.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df_bop.columns]).show()

Null value counts:
+-------+--------------------+---------+----+---------+-----------+-----+------+
|COUNTRY|BOP_ACCOUNTING_ENTRY|INDICATOR|UNIT|FREQUENCY|TIME_PERIOD|value| index|
+-------+--------------------+---------+----+---------+-----------+-----+------+
|      0|                   0|        0|   0|        0|          0|    0|180804|
+-------+--------------------+---------+----+---------+-----------+-----+------+

+-------+--------------------+---------+----+---------+-----------+-----+------+
|COUNTRY|BOP_ACCOUNTING_ENTRY|INDICATOR|UNIT|FREQUENCY|TIME_PERIOD|value| index|
+-------+--------------------+---------+----+---------+-----------+-----+------+
|      0|                   0|        0|   0|        0|          0|    0|180804|
+-------+--------------------+---------+----+---------+-----------+-----+------+



In [24]:
# Statistics on value column
print("Value column statistics:")
df_bop.select("value").describe().show()

Value column statistics:
+-------+--------------------+
|summary|               value|
+-------+--------------------+
|  count|              180804|
|   mean| 8.002790936956046E9|
| stddev|4.131887512070207E10|
|    min|-6.73055075813792...|
|    max|         1.158498E12|
+-------+--------------------+

+-------+--------------------+
|summary|               value|
+-------+--------------------+
|  count|              180804|
|   mean| 8.002790936956046E9|
| stddev|4.131887512070207E10|
|    min|-6.73055075813792...|
|    max|         1.158498E12|
+-------+--------------------+



In [25]:
# Value distribution by unit
print("Value statistics by unit:")
df_bop.groupBy("UNIT") \
    .agg(
        F.count("value").alias("count"),
        F.round(F.avg("value"), 2).alias("avg_value"),
        F.round(F.min("value"), 2).alias("min_value"),
        F.round(F.max("value"), 2).alias("max_value")
    ) \
    .show()

Value statistics by unit:
+----+------+---------------+-------------------+-----------+
|UNIT| count|      avg_value|          min_value|  max_value|
+----+------+---------------+-------------------+-----------+
| USD|180804|8.00279093696E9|-6.7305507581379E11|1.158498E12|
+----+------+---------------+-------------------+-----------+

+----+------+---------------+-------------------+-----------+
|UNIT| count|      avg_value|          min_value|  max_value|
+----+------+---------------+-------------------+-----------+
| USD|180804|8.00279093696E9|-6.7305507581379E11|1.158498E12|
+----+------+---------------+-------------------+-----------+



## 9. Time Period Analysis

In [26]:
# Extract year from TIME_PERIOD
df_bop_with_year = df_bop.withColumn(
    "year", 
    F.substring(F.col("TIME_PERIOD"), 1, 4).cast("int")
)

# Records by year
print("Records by year:")
df_bop_with_year.groupBy("year") \
    .count() \
    .orderBy("year") \
    .show(10)

Records by year:
+----+-----+
|year|count|
+----+-----+
|2000| 5888|
|2001| 5916|
|2002| 6232|
|2003| 6260|
|2004| 6292|
|2005| 6804|
|2006| 6870|
|2007| 6873|
|2008| 6934|
|2009| 6948|
+----+-----+
only showing top 10 rows

+----+-----+
|year|count|
+----+-----+
|2000| 5888|
|2001| 5916|
|2002| 6232|
|2003| 6260|
|2004| 6292|
|2005| 6804|
|2006| 6870|
|2007| 6873|
|2008| 6934|
|2009| 6948|
+----+-----+
only showing top 10 rows



## 10. Summary Report

In [27]:
# Generate summary report
total_records = df_bop.count()
total_countries = df_bop.select("COUNTRY").distinct().count()
total_indicators = df_bop.select("INDICATOR").distinct().count()
date_range = df_bop.select(
    F.min("TIME_PERIOD").alias("min_date"),
    F.max("TIME_PERIOD").alias("max_date")
).collect()[0]

print("="*60)
print("BOP DATA ANALYSIS SUMMARY")
print("="*60)
print(f"Total Records: {total_records:,}")
print(f"Total Countries: {total_countries}")
print(f"Total Indicators: {total_indicators}")
print(f"Date Range: {date_range['min_date']} to {date_range['max_date']}")
print("="*60)

BOP DATA ANALYSIS SUMMARY
Total Records: 180,804
Total Countries: 51
Total Indicators: 21
Date Range: 2000-Q1 to 2025-Q2


## 11. Cleanup

In [28]:
# Stop Spark session
spark_config.stop_session()
print("Analysis completed successfully!")

INFO:config:Spark Session finalizada


Analysis completed successfully!
